# Setup

This is following [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for pandas 1.4.3

In [1]:
import numpy as np
import pandas as pd

# Object creation

In [2]:
# Series - pass a list to pd.Series
s1 = pd.Series([1, 2, 3, np.nan, 5])

In [3]:
s1

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

In [4]:
# create an index to use in the dataframe
# default frequency is D (day), so creates a DatetimeIndex of 10 days
dates = pd.date_range('20220619', periods=10)
dates

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

just in case you are wondering, here's [all the "offset aliases"](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases) for Datetime

In [5]:
# Dataframe - pass a Numpy array
# create a 10 row, 4 col random number array, index by dates, give some column names
df = pd.DataFrame(np.random.randn(10, 4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-21,2.782728,-0.813920,1.421262,0.897830
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-23,-1.053229,0.618682,0.026133,0.208777
2022-06-24,-0.324889,0.016263,-0.588217,0.129666
2022-06-25,-0.322421,1.040214,-0.492945,0.803259
2022-06-26,0.044602,-0.515809,0.178538,-1.294326
2022-06-27,-1.284847,0.014703,1.330946,-0.123998
2022-06-28,1.906451,-1.147632,0.150863,0.392640


In [6]:
# Create a dataframe by passing a dictionary of objects
# where each object can be converted into a series-like structure
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20220619"),
        "C": pd.Series(1, index=list(range(6)), dtype="float32"),
        "D": np.array([3]*6, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train", "test", "train"]),
        "F": list("foofoo"),
        "G": "foo"
    }
)

df2

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


In [7]:
# the datatype of each of the columns would be different
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
G            object
dtype: object

# Viewing Data

In [8]:
# top of the dataframe
df.head()

,A,B,C,D
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-21,2.782728,-0.813920,1.421262,0.897830
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-23,-1.053229,0.618682,0.026133,0.208777


In [9]:
# bottom 3 records of the dataframe
df.tail(3)

,A,B,C,D
2022-06-26,0.044602,-0.515809,0.178538,-1.294326
2022-06-27,-1.284847,0.014703,1.330946,-0.123998
2022-06-28,1.906451,-1.147632,0.150863,0.392640


In [10]:
# index of the df
df.index

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

In [11]:
# columns of the df
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [12]:
# .to_numpy gives a NumPy representation of the dataframe
# this is expensive if all columns are of different data type
df.to_numpy()

array([[-0.31573043, -0.43637993, -0.86734854, -1.59647355],
       [ 0.74257854, -0.4117314 ,  0.16571812,  0.36235218],
       [ 2.78272753, -0.81392024,  1.42126167,  0.89783038],
       [ 0.73379493, -1.00702298, -1.40360109, -0.90655556],
       [-1.05322933,  0.61868223,  0.02613302,  0.20877676],
       [-0.32488941,  0.01626294, -0.58821689,  0.12966645],
       [-0.3224214 ,  1.04021371, -0.49294527,  0.80325926],
       [ 0.04460237, -0.51580937,  0.1785376 , -1.29432607],
       [-1.2848469 ,  0.01470275,  1.3309457 , -0.12399828],
       [ 1.90645111, -1.14763162,  0.15086283,  0.3926395 ]])

In [13]:
df2.to_numpy()

array([[1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo']], dtype=object)

In [14]:
# quick summary stats
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.290904,-0.264263,-0.007865,-0.112683
std,1.277718,0.698138,0.893492,0.864834
min,-1.284847,-1.147632,-1.403601,-1.596474
25%,-0.324272,-0.739393,-0.564399,-0.710916
50%,-0.135564,-0.424056,0.088498,0.169222
75%,0.740383,0.015873,0.175333,0.385068
max,2.782728,1.040214,1.421262,0.897830


1. **count** = Count number of non-NA/null observations
1. **max** = Maximum of the values in the object
1. **min** = Minimum of the values in the object
1. **mean** = Mean of the values
1. **std** = Standard deviation of the observations
1. **25%** = Default lower percentile
1. **50%** = 50 percentile - same as the median 
1. **75%** = Default upper percentile

In [15]:
# change the percentiles
df.describe(percentiles=[.1, .5, .9])

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.290904,-0.264263,-0.007865,-0.112683
std,1.277718,0.698138,0.893492,0.864834
min,-1.284847,-1.147632,-1.403601,-1.596474
10%,-1.076391,-1.021084,-0.920974,-1.324541
50%,-0.135564,-0.424056,0.088498,0.169222
90%,1.994079,0.660835,1.339977,0.812716
max,2.782728,1.040214,1.421262,0.897830


In [16]:
# E, F, G in d2 are not numeric, so do not come up in describe()
df2.describe()

,A,C,D
count,6.0,6.0,6.0
mean,1.0,1.0,3.0
std,0.0,0.0,0.0
min,1.0,1.0,3.0
25%,1.0,1.0,3.0
50%,1.0,1.0,3.0
75%,1.0,1.0,3.0
max,1.0,1.0,3.0


In [17]:
# transpose the data
# turn rows to columns and vice versa
df.T

,2022-06-19,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28
A,-0.315730,0.742579,2.782728,0.733795,-1.053229,-0.324889,-0.322421,0.044602,-1.284847,1.906451
B,-0.436380,-0.411731,-0.813920,-1.007023,0.618682,0.016263,1.040214,-0.515809,0.014703,-1.147632
C,-0.867349,0.165718,1.421262,-1.403601,0.026133,-0.588217,-0.492945,0.178538,1.330946,0.150863
D,-1.596474,0.362352,0.897830,-0.906556,0.208777,0.129666,0.803259,-1.294326,-0.123998,0.392640


### [Axis in pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_axis.html?highlight=set_ax#pandas.DataFrame.set_axis)

```axis{0 or ‘index’, 1 or ‘columns’}```

Always remember that when you specify:
* ```axis=0``` or ```axis='index'``` you indicate that the operation should be along the _index_ (aka across the rows). More often than not indexes will go from top to bottom (vertically), but sometimes they may not (for e.g. if you use columns as index or when you have hierarchical or multi-index data).
* ```axis=1``` or ```axis='columns'``` indicates the operation is along the columns

When in doubt use the explicit version ```'index'``` or ```'columns'``` instead of ```0``` or ```1```

In [18]:
# sort along the axis - 1 = horizontal
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2022-06-19,-1.596474,-0.867349,-0.436380,-0.315730
2022-06-20,0.362352,0.165718,-0.411731,0.742579
2022-06-21,0.897830,1.421262,-0.813920,2.782728
2022-06-22,-0.906556,-1.403601,-1.007023,0.733795
2022-06-23,0.208777,0.026133,0.618682,-1.053229
2022-06-24,0.129666,-0.588217,0.016263,-0.324889
2022-06-25,0.803259,-0.492945,1.040214,-0.322421
2022-06-26,-1.294326,0.178538,-0.515809,0.044602
2022-06-27,-0.123998,1.330946,0.014703,-1.284847
2022-06-28,0.392640,0.150863,-1.147632,1.906451


In [19]:
# sort along the axis - 0 = vertical
df.sort_index(axis=0, ascending=False)

,A,B,C,D
2022-06-28,1.906451,-1.147632,0.150863,0.392640
2022-06-27,-1.284847,0.014703,1.330946,-0.123998
2022-06-26,0.044602,-0.515809,0.178538,-1.294326
2022-06-25,-0.322421,1.040214,-0.492945,0.803259
2022-06-24,-0.324889,0.016263,-0.588217,0.129666
2022-06-23,-1.053229,0.618682,0.026133,0.208777
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-21,2.782728,-0.813920,1.421262,0.897830
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474


In [20]:
# sort ascending by values in a column
df.sort_values(by="A")

,A,B,C,D
2022-06-27,-1.284847,0.014703,1.330946,-0.123998
2022-06-23,-1.053229,0.618682,0.026133,0.208777
2022-06-24,-0.324889,0.016263,-0.588217,0.129666
2022-06-25,-0.322421,1.040214,-0.492945,0.803259
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474
2022-06-26,0.044602,-0.515809,0.178538,-1.294326
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-28,1.906451,-1.147632,0.150863,0.392640
2022-06-21,2.782728,-0.813920,1.421262,0.897830


In [21]:
# sort by non-numerical values
df2.sort_values(by="F", ascending=False)

,A,B,C,D,E,F,G
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo


In [22]:
# sort by two or more columns
df2.sort_values(by=["F", "E"])

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


# Selection

For production prefer the following instead of other data access methods (typical python methods like ["col"] or [a:b] slices etc.):

```.at, .iat, .loc and .iloc.```

## Getting

In [23]:
# selecting a single column returns a Series object
df["A"]

2022-06-19   -0.315730
2022-06-20    0.742579
2022-06-21    2.782728
2022-06-22    0.733795
2022-06-23   -1.053229
2022-06-24   -0.324889
2022-06-25   -0.322421
2022-06-26    0.044602
2022-06-27   -1.284847
2022-06-28    1.906451
Freq: D, Name: A, dtype: float64

In [24]:
# selecting a slice
df[1:5]

,A,B,C,D
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-21,2.782728,-0.813920,1.421262,0.897830
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-23,-1.053229,0.618682,0.026133,0.208777


## Selection by label

In [25]:
# selecting based on a label
df.loc[dates[0]]

A   -0.315730
B   -0.436380
C   -0.867349
D   -1.596474
Name: 2022-06-19 00:00:00, dtype: float64

In [26]:
# select on a multi-axis by lable
# I honestly do not know what the comment above means...help!
df.loc[:, ["A", "B"]]

,A,B
2022-06-19,-0.315730,-0.436380
2022-06-20,0.742579,-0.411731
2022-06-21,2.782728,-0.813920
2022-06-22,0.733795,-1.007023
2022-06-23,-1.053229,0.618682
2022-06-24,-0.324889,0.016263
2022-06-25,-0.322421,1.040214
2022-06-26,0.044602,-0.515809
2022-06-27,-1.284847,0.014703
2022-06-28,1.906451,-1.147632


In [27]:
# specific index value results in reduction of dimensions
res = df.loc["2022-06-20"]
print(res)
print("res.shape = ", res.shape, " vs. df.shape = ", df.shape)

A    0.742579
B   -0.411731
C    0.165718
D    0.362352
Name: 2022-06-20 00:00:00, dtype: float64
res.shape =  (4,)  vs. df.shape =  (10, 4)


In [28]:
# get to a specific scalar:
#
# method one
df.loc[dates[0], "A"]

-0.3157304282588729

In [29]:
#
# method two (slightly faster than method one)
df.at[dates[0], "A"]

-0.3157304282588729

## Selection by position

In [30]:
df.iloc[2]

A    2.782728
B   -0.813920
C    1.421262
D    0.897830
Name: 2022-06-21 00:00:00, dtype: float64

In [31]:
# slices - similar to NumPy / Python - [row:slice, col:slice]
df.iloc[1:5, 0:2]

,A,B
2022-06-20,0.742579,-0.411731
2022-06-21,2.782728,-0.813920
2022-06-22,0.733795,-1.007023
2022-06-23,-1.053229,0.618682


In [32]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
df.iloc[[0, 1, 2, 6], [0, 2]]

,A,C
2022-06-19,-0.315730,-0.867349
2022-06-20,0.742579,0.165718
2022-06-21,2.782728,1.421262
2022-06-25,-0.322421,-0.492945


In [33]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
# change the order of columns, repeact a column
df.iloc[[0, 1, 2, 6], [2, 1, 0, 2]]

,C,B,A,C
2022-06-19,-0.867349,-0.436380,-0.315730,-0.867349
2022-06-20,0.165718,-0.411731,0.742579,0.165718
2022-06-21,1.421262,-0.813920,2.782728,1.421262
2022-06-25,-0.492945,1.040214,-0.322421,-0.492945


In [34]:
# slice rows explicitly, keep all columns
df.iloc[[1, 2], :]

,A,B,C,D
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-21,2.782728,-0.813920,1.421262,0.897830


In [35]:
# slice columns, keep all rows
df.iloc[:, [2, 3]]

,C,D
2022-06-19,-0.867349,-1.596474
2022-06-20,0.165718,0.362352
2022-06-21,1.421262,0.897830
2022-06-22,-1.403601,-0.906556
2022-06-23,0.026133,0.208777
2022-06-24,-0.588217,0.129666
2022-06-25,-0.492945,0.803259
2022-06-26,0.178538,-1.294326
2022-06-27,1.330946,-0.123998
2022-06-28,0.150863,0.392640


In [36]:
# everything, because you can
df.iloc[:, :]

,A,B,C,D
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-21,2.782728,-0.813920,1.421262,0.897830
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-23,-1.053229,0.618682,0.026133,0.208777
2022-06-24,-0.324889,0.016263,-0.588217,0.129666
2022-06-25,-0.322421,1.040214,-0.492945,0.803259
2022-06-26,0.044602,-0.515809,0.178538,-1.294326
2022-06-27,-1.284847,0.014703,1.330946,-0.123998
2022-06-28,1.906451,-1.147632,0.150863,0.392640


In [37]:
# get to a scalar (2 methods, just like before)
#
# method one: use iloc
df.iloc[1, 2]

0.16571812449917717

In [38]:
#
# method two: use iat
df.iat[1, 2]

0.16571812449917717

## Boolean Indexing

In [39]:
# use a value found in a single col to get data
df[df["A"] > 0]

,A,B,C,D
2022-06-20,0.742579,-0.411731,0.165718,0.362352
2022-06-21,2.782728,-0.813920,1.421262,0.897830
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556
2022-06-26,0.044602,-0.515809,0.178538,-1.294326
2022-06-28,1.906451,-1.147632,0.150863,0.392640


In [40]:
# boolean across the entire DF - vals that don't match go NaN
df[df > 0]

,A,B,C,D
2022-06-19,NaN,NaN,NaN,NaN
2022-06-20,0.742579,NaN,0.165718,0.362352
2022-06-21,2.782728,NaN,1.421262,0.897830
2022-06-22,0.733795,NaN,NaN,NaN
2022-06-23,NaN,0.618682,0.026133,0.208777
2022-06-24,NaN,0.016263,NaN,0.129666
2022-06-25,NaN,1.040214,NaN,0.803259
2022-06-26,0.044602,NaN,0.178538,NaN
2022-06-27,NaN,0.014703,1.330946,NaN
2022-06-28,1.906451,NaN,0.150863,0.392640


In [41]:
# add another column
df11 = df.copy()
df11["E"] = ["one", "two", "three", "four",
             "two", "five", "one", "two", "three", "four"]
df11

,A,B,C,D,E
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,one
2022-06-20,0.742579,-0.411731,0.165718,0.362352,two
2022-06-21,2.782728,-0.813920,1.421262,0.897830,three
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,four
2022-06-23,-1.053229,0.618682,0.026133,0.208777,two
2022-06-24,-0.324889,0.016263,-0.588217,0.129666,five
2022-06-25,-0.322421,1.040214,-0.492945,0.803259,one
2022-06-26,0.044602,-0.515809,0.178538,-1.294326,two
2022-06-27,-1.284847,0.014703,1.330946,-0.123998,three
2022-06-28,1.906451,-1.147632,0.150863,0.392640,four


In [42]:
# the isin() query - basically the in clause
df11[df11["E"].isin(["two", "five"])]

,A,B,C,D,E
2022-06-20,0.742579,-0.411731,0.165718,0.362352,two
2022-06-23,-1.053229,0.618682,0.026133,0.208777,two
2022-06-24,-0.324889,0.016263,-0.588217,0.129666,five
2022-06-26,0.044602,-0.515809,0.178538,-1.294326,two


## Setting values

In [43]:
# matching indexes auto-aligns values
s1 = pd.Series(range(11, 21), index=pd.date_range('20220619', periods=10))
s1

2022-06-19    11
2022-06-20    12
2022-06-21    13
2022-06-22    14
2022-06-23    15
2022-06-24    16
2022-06-25    17
2022-06-26    18
2022-06-27    19
2022-06-28    20
Freq: D, dtype: int64

In [44]:
df["F"] = s1
df

,A,B,C,D,F
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11
2022-06-20,0.742579,-0.411731,0.165718,0.362352,12
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14
2022-06-23,-1.053229,0.618682,0.026133,0.208777,15
2022-06-24,-0.324889,0.016263,-0.588217,0.129666,16
2022-06-25,-0.322421,1.040214,-0.492945,0.803259,17
2022-06-26,0.044602,-0.515809,0.178538,-1.294326,18
2022-06-27,-1.284847,0.014703,1.330946,-0.123998,19
2022-06-28,1.906451,-1.147632,0.150863,0.392640,20


In [45]:
# setting values by label and position
# first let's make a quick copy
df12 = df.copy()
df12

,A,B,C,D,F
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11
2022-06-20,0.742579,-0.411731,0.165718,0.362352,12
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14
2022-06-23,-1.053229,0.618682,0.026133,0.208777,15
2022-06-24,-0.324889,0.016263,-0.588217,0.129666,16
2022-06-25,-0.322421,1.040214,-0.492945,0.803259,17
2022-06-26,0.044602,-0.515809,0.178538,-1.294326,18
2022-06-27,-1.284847,0.014703,1.330946,-0.123998,19
2022-06-28,1.906451,-1.147632,0.150863,0.392640,20


In [46]:
# set by label
df12.at[dates[0], "A"] = 0
# set by position
df12.iat[0, 1] = 0
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-0.867349,-1.596474,11
2022-06-20,0.742579,-0.411731,0.165718,0.362352,12
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14
2022-06-23,-1.053229,0.618682,0.026133,0.208777,15
2022-06-24,-0.324889,0.016263,-0.588217,0.129666,16
2022-06-25,-0.322421,1.040214,-0.492945,0.803259,17
2022-06-26,0.044602,-0.515809,0.178538,-1.294326,18
2022-06-27,-1.284847,0.014703,1.330946,-0.123998,19
2022-06-28,1.906451,-1.147632,0.150863,0.392640,20


In [47]:
# kinda bigger replacement
df12.loc[:, "D"] = np.array([5]*len(df))
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-0.867349,5,11
2022-06-20,0.742579,-0.411731,0.165718,5,12
2022-06-21,2.782728,-0.813920,1.421262,5,13
2022-06-22,0.733795,-1.007023,-1.403601,5,14
2022-06-23,-1.053229,0.618682,0.026133,5,15
2022-06-24,-0.324889,0.016263,-0.588217,5,16
2022-06-25,-0.322421,1.040214,-0.492945,5,17
2022-06-26,0.044602,-0.515809,0.178538,5,18
2022-06-27,-1.284847,0.014703,1.330946,5,19
2022-06-28,1.906451,-1.147632,0.150863,5,20


In [48]:
# setting values using a boolean selection (aka where clause)
df12[df12 > 0] = -df12
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-0.867349,-5,-11
2022-06-20,-0.742579,-0.411731,-0.165718,-5,-12
2022-06-21,-2.782728,-0.813920,-1.421262,-5,-13
2022-06-22,-0.733795,-1.007023,-1.403601,-5,-14
2022-06-23,-1.053229,-0.618682,-0.026133,-5,-15
2022-06-24,-0.324889,-0.016263,-0.588217,-5,-16
2022-06-25,-0.322421,-1.040214,-0.492945,-5,-17
2022-06-26,-0.044602,-0.515809,-0.178538,-5,-18
2022-06-27,-1.284847,-0.014703,-1.330946,-5,-19
2022-06-28,-1.906451,-1.147632,-0.150863,-5,-20


# Missing Data

### _reindex_
change/add/delete index on a specified axis, returns a new dataframe

In [49]:
df13 = df.reindex(index=dates[0:4], columns=list(df.columns)+["G"])
df13

,A,B,C,D,F,G
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11,NaN
2022-06-20,0.742579,-0.411731,0.165718,0.362352,12,NaN
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,NaN
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,NaN


### handling missing data
1. _```np.nan```_
1. _```pandas.isna()```_
1. ```df.dropna()```
1. ```df.fillna()```

In [50]:
# missing data in pandas is np.nan
df13.iat[1, 0] = np.nan
df13.iloc[1:, 5] = np.random.randint(1)
df13

,A,B,C,D,F,G
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11,NaN
2022-06-20,NaN,-0.411731,0.165718,0.362352,12,0.0
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.0
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.0


In [51]:
# get a boolean mask where values are NaN
df131 = pd.isna(df13)
df131

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [52]:
# or just
pd.isna(df13)

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [53]:
# the original is still there
df13

,A,B,C,D,F,G
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11,NaN
2022-06-20,NaN,-0.411731,0.165718,0.362352,12,0.0
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.0
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.0


In [54]:
# we are going to drop / replace values now, let's make a couple of copies of the dataframe
df132 = df13.copy()
df132

,A,B,C,D,F,G
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11,NaN
2022-06-20,NaN,-0.411731,0.165718,0.362352,12,0.0
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.0
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.0


In [55]:
# drop rows/columns that have missing data
# by default it returns a new dataframe, you may want to specify inplace=True for modifying current dataframe:
df_no_na = df132.dropna(how="any")
# how=‘any’ : If any NA values are present, drop that row or column.
# how=‘all’ : If all values are NA, drop that row or column.

In [56]:
# all rows/cols with missing data stripped
df_no_na

,A,B,C,D,F,G
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.0
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.0


In [57]:
# original still intact
df132

,A,B,C,D,F,G
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11,NaN
2022-06-20,NaN,-0.411731,0.165718,0.362352,12,0.0
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.0
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.0


In [58]:
# drop missing data from original
df132.dropna(how="any", inplace=True)
df132

,A,B,C,D,F,G
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.0
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.0


In [59]:
# fill missing data
df133 = df13.copy()
df133.fillna(np.pi*1000)

,A,B,C,D,F,G
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11,3141.592654
2022-06-20,3141.592654,-0.411731,0.165718,0.362352,12,0.000000
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13,0.000000
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14,0.000000


# Align and Join

There needs to be a bigger notebook for this topic.
You need to know there's "joins" in Pandas just like in the SQL world, 
like join and left join and right join and inner and outer and all that...

## Create two datasets with _mismatched_ indexes

In [60]:
# date range indexes
idx1 = pd.date_range('2022-01-01',periods = 10)
# 2022-01-01', '2022-01-02' don't exist in idx2
# '2022-01-11', '2022-01-12' don't exist in idx1
idx2 = pd.date_range('2022-01-03',periods = 10)

In [61]:
# dataframes from indexes
d1 = pd.DataFrame(index = idx1, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

d2 = pd.DataFrame(index = idx2, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

## Quick Aside: Rendering two dataframes side-by-side

The trick of rendering 2 dataframes side-by-side was from [this](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side) stackoverflow question.

In [84]:
# we gon need to display both data frames side by side, so...
from IPython.display import display_html 
def render_df_side_by_side(a, b, a_title = "", b_title = ""):
    a_styler = a.style.set_table_attributes("style='display:inline'").set_caption(a_title)
    b_styler = b.style.set_table_attributes("style='display:inline'").set_caption(b_title)
    display_html(a_styler._repr_html_()+b_styler._repr_html_(), raw=True)

In [85]:
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.011493,23
2022-01-02 00:00:00,0.495092,12
2022-01-03 00:00:00,0.249666,21
2022-01-04 00:00:00,0.713133,11
2022-01-05 00:00:00,0.982975,24
2022-01-06 00:00:00,0.369567,22
2022-01-07 00:00:00,0.275929,5
2022-01-08 00:00:00,0.372111,4
2022-01-09 00:00:00,0.202119,24
2022-01-10 00:00:00,0.415255,1


Table rendering/styling options is [a bigger discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#) to be tackled later.

## .add, .sub, .mul, .div etc.

flexible wrappers (```add, sub, mul, div, mod, pow```) to arithmetic operators: ```+, -, *, /, //, %, **```

In [64]:
# the +, -, *, /, //, %, ** operations align indexes to create a UNION of indexes
d3 = d1+d2
d4 = d1.sub(d2) #equivalent to d1-d2
d5 = d1*d2
d6 = d1.div(d2)

In [65]:
# indexs not present in the other dataframe get a NaN
d3.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.634239,38.000000
2022-01-04 00:00:00,1.262588,18.000000
2022-01-05 00:00:00,1.428408,30.000000
2022-01-06 00:00:00,1.011450,29.000000
2022-01-07 00:00:00,0.723338,8.000000
2022-01-08 00:00:00,1.236861,14.000000
2022-01-09 00:00:00,0.635790,31.000000
2022-01-10 00:00:00,0.618221,22.000000


In [66]:
# sub
d4.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,-0.134906,4.000000
2022-01-04 00:00:00,0.163678,4.000000
2022-01-05 00:00:00,0.537542,18.000000
2022-01-06 00:00:00,-0.272316,15.000000
2022-01-07 00:00:00,-0.171480,2.000000
2022-01-08 00:00:00,-0.492639,-6.000000
2022-01-09 00:00:00,-0.231552,17.000000
2022-01-10 00:00:00,0.212289,-20.000000


In [67]:
# mul
d5.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.096015,357.000000
2022-01-04 00:00:00,0.391834,77.000000
2022-01-05 00:00:00,0.437850,144.000000
2022-01-06 00:00:00,0.237219,154.000000
2022-01-07 00:00:00,0.123453,15.000000
2022-01-08 00:00:00,0.321783,40.000000
2022-01-09 00:00:00,0.087653,168.000000
2022-01-10 00:00:00,0.084283,21.000000


In [68]:
# div
d6.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.649205,1.235294
2022-01-04 00:00:00,1.297892,1.571429
2022-01-05 00:00:00,2.206786,4.000000
2022-01-06 00:00:00,0.575754,3.142857
2022-01-07 00:00:00,0.616726,1.666667
2022-01-08 00:00:00,0.430310,0.400000
2022-01-09 00:00:00,0.466066,3.428571
2022-01-10 00:00:00,2.045936,0.047619


### removing the NaN values from the resultant dataframe

In [69]:
# the + operation is UNION of indexes
d31 = d1+d2
# indexs not present in the other dataframe get a NaN
# do you can obvs remove nan values like before
d3.dropna().style.highlight_null(null_color='red')

,A,B
2022-01-03 00:00:00,0.634239,38.000000
2022-01-04 00:00:00,1.262588,18.000000
2022-01-05 00:00:00,1.428408,30.000000
2022-01-06 00:00:00,1.011450,29.000000
2022-01-07 00:00:00,0.723338,8.000000
2022-01-08 00:00:00,1.236861,14.000000
2022-01-09 00:00:00,0.635790,31.000000
2022-01-10 00:00:00,0.618221,22.000000


## df.align()

Pandas [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) works to join the dataframes in interesting ways.

In [70]:
# align returns two dataframes - left and right as a result of a join
# default join is outer - the union of indices will be used as index for left and right
left, right = d1.align(d2, join='outer')
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.011493,23.000000
2022-01-02 00:00:00,0.495092,12.000000
2022-01-03 00:00:00,0.249666,21.000000
2022-01-04 00:00:00,0.713133,11.000000
2022-01-05 00:00:00,0.982975,24.000000
2022-01-06 00:00:00,0.369567,22.000000
2022-01-07 00:00:00,0.275929,5.000000
2022-01-08 00:00:00,0.372111,4.000000
2022-01-09 00:00:00,0.202119,24.000000
2022-01-10 00:00:00,0.415255,1.000000


In [71]:
# inner join
left, right = d1.align(d2, join='inner')
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.011493,23
2022-01-02 00:00:00,0.495092,12
2022-01-03 00:00:00,0.249666,21
2022-01-04 00:00:00,0.713133,11
2022-01-05 00:00:00,0.982975,24
2022-01-06 00:00:00,0.369567,22
2022-01-07 00:00:00,0.275929,5
2022-01-08 00:00:00,0.372111,4
2022-01-09 00:00:00,0.202119,24
2022-01-10 00:00:00,0.415255,1


In [72]:
# we can fill the NaNs with a certain value if needed
# let's fill with -100 so we can see it clearly
left, right = d1.align(d2, join='outer', fill_value = -100) 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.011493,23
2022-01-02 00:00:00,0.495092,12
2022-01-03 00:00:00,0.249666,21
2022-01-04 00:00:00,0.713133,11
2022-01-05 00:00:00,0.982975,24
2022-01-06 00:00:00,0.369567,22
2022-01-07 00:00:00,0.275929,5
2022-01-08 00:00:00,0.372111,4
2022-01-09 00:00:00,0.202119,24
2022-01-10 00:00:00,0.415255,1


In [73]:
# left join: left index is added to right, but not the other way
left, right = d1.align(d2, join='left') 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.011493,23
2022-01-02 00:00:00,0.495092,12
2022-01-03 00:00:00,0.249666,21
2022-01-04 00:00:00,0.713133,11
2022-01-05 00:00:00,0.982975,24
2022-01-06 00:00:00,0.369567,22
2022-01-07 00:00:00,0.275929,5
2022-01-08 00:00:00,0.372111,4
2022-01-09 00:00:00,0.202119,24
2022-01-10 00:00:00,0.415255,1


In [74]:
# right join: right index is added to left, but not the other way
left, right = d1.align(d2, join='right') 
render_df_side_by_side(left, right)

,A,B
2022-01-03 00:00:00,0.249666,21.000000
2022-01-04 00:00:00,0.713133,11.000000
2022-01-05 00:00:00,0.982975,24.000000
2022-01-06 00:00:00,0.369567,22.000000
2022-01-07 00:00:00,0.275929,5.000000
2022-01-08 00:00:00,0.372111,4.000000
2022-01-09 00:00:00,0.202119,24.000000
2022-01-10 00:00:00,0.415255,1.000000
2022-01-11 00:00:00,nan,nan
2022-01-12 00:00:00,nan,nan


There's more to explore in align, and it can get confusing, so beginners be careful, take time and try to work out the result before you execute to build intuition.

# Operations on data

## Stats

Operations in general *exclude* missing data

In [75]:
# arithmetic mean, for each column (axis = 0)
df.mean()

A     0.290904
B    -0.264263
C    -0.007865
D    -0.112683
F    15.500000
dtype: float64

In [76]:
# mean across a row (axis = 1)
df.mean(1)

2022-06-19    1.556814
2022-06-20    2.571783
2022-06-21    3.457580
2022-06-22    2.283323
2022-06-23    2.960073
2022-06-24    3.046565
2022-06-25    3.605621
2022-06-26    3.282601
2022-06-27    3.787361
2022-06-28    4.260464
Freq: D, dtype: float64

## Apply

In [79]:
# apply custom lambdas
df.apply(lambda x: x.max()-x.min())

A    4.067574
B    2.187845
C    2.824863
D    2.494304
F    9.000000
dtype: float64

In [81]:
df

,A,B,C,D,F
2022-06-19,-0.315730,-0.436380,-0.867349,-1.596474,11
2022-06-20,0.742579,-0.411731,0.165718,0.362352,12
2022-06-21,2.782728,-0.813920,1.421262,0.897830,13
2022-06-22,0.733795,-1.007023,-1.403601,-0.906556,14
2022-06-23,-1.053229,0.618682,0.026133,0.208777,15
2022-06-24,-0.324889,0.016263,-0.588217,0.129666,16
2022-06-25,-0.322421,1.040214,-0.492945,0.803259,17
2022-06-26,0.044602,-0.515809,0.178538,-1.294326,18
2022-06-27,-1.284847,0.014703,1.330946,-0.123998,19
2022-06-28,1.906451,-1.147632,0.150863,0.392640,20


In [91]:
# each subsequent value is a sum of all values before it in the respective column
cumsum_df = df.apply(np.cumsum)
render_df_side_by_side(df, cumsum_df, "OG Dataframe", "Cumulative Sum")

,A,B,C,D,F
2022-06-19 00:00:00,-0.315730,-0.436380,-0.867349,-1.596474,11
2022-06-20 00:00:00,0.742579,-0.411731,0.165718,0.362352,12
2022-06-21 00:00:00,2.782728,-0.813920,1.421262,0.897830,13
2022-06-22 00:00:00,0.733795,-1.007023,-1.403601,-0.906556,14
2022-06-23 00:00:00,-1.053229,0.618682,0.026133,0.208777,15
2022-06-24 00:00:00,-0.324889,0.016263,-0.588217,0.129666,16
2022-06-25 00:00:00,-0.322421,1.040214,-0.492945,0.803259,17
2022-06-26 00:00:00,0.044602,-0.515809,0.178538,-1.294326,18
2022-06-27 00:00:00,-1.284847,0.014703,1.330946,-0.123998,19
2022-06-28 00:00:00,1.906451,-1.147632,0.150863,0.392640,20


## Histogramming

Frequencies, 'nuff said

```value_counts()```

In [93]:
series1 = pd.Series(np.random.randint(0,5, size = 25))
series1.value_counts()

4    7
3    6
1    5
2    4
0    3
dtype: int64

## String Methods

In [98]:
# build a series
some_string = "SERIes and Index are EqUIppeD WITh A seT Of stRINg PROCESSING METHoDs In tHe ```sTR``` attrIBute THAT MakE IT eASy To oPerATE ON EACh ELEmeNt OF The aRrAY. NOtE thAt PAtTERN-maTching in ```sTR``` gENeralLY UsES reGUlAr eXpreSsiONs bY DEfault (aND In Some cases AlwayS uSEs tHeM)."
str_series = pd.Series(some_string.split()) # split splits on whitespace by default

In [117]:
# manipulating the case - lower, upper etc.
# also the length of each string, split, replace, yada yada...
# very important when cleaning data (column names may have stupid whitespace, bad case, spacing etc.)
low = str_series.str.lower()
up = str_series.str.upper()

low_up_df = pd.DataFrame(
    {
        'og': str_series,
        'length': str_series.str.len(),
        'low': low,
        'up': up,
        'split_low_on_i': low.str.split('i'),
        'replace_a_with_star_in_low': low.str.replace('a','*', case=False, regex=True) # yeah, you can mess with regular expns
    })

low_up_df.tail(10)

,og,length,low,up,split_low_on_i,replace_a_with_star_in_low
36,eXpreSsiONs,11,expressions,EXPRESSIONS,"[express, ons]",expressions
37,bY,2,by,BY,[by],by
38,DEfault,7,default,DEFAULT,[default],def*ult
39,(aND,4,(and,(AND,[(and],(*nd
40,In,2,in,IN,"[, n]",in
41,Some,4,some,SOME,[some],some
42,cases,5,cases,CASES,[cases],c*ses
43,AlwayS,6,always,ALWAYS,[always],*lw*ys
44,uSEs,4,uses,USES,[uses],uses
45,tHeM).,6,them).,THEM).,[them).],them).


See [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-string-methods) for more string operations.